In [0]:
import io
import requests

import pandas as pd
import numpy as np

from scipy.stats import uniform

from sklearn.linear_model import ElasticNet, PassiveAggressiveRegressor, LinearRegression, ARDRegression, ElasticNetCV, HuberRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn import ensemble

Uploading competition data

In [0]:
url_X_train = "https://storage.googleapis.com/kagglesdsdata/competitions/20137/1112534/trainX.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1589791225&Signature=sX0eN%2FN3z%2Fqj0zu%2BANUYM%2F%2Fls3fa4UFao9UeV7IX%2BVnhlWbB5LtrO%2BsdZ6XPd63O%2F4FaqSwL8%2Ff4nxJZQ9iY3QcgjivcbqUZ9krk7pEFTXDekyCFzmMglbNgSXUFXLXhcCLZP4qR1yr%2Ba72edgKLY3DXYV%2Fd1fTFjtTXkOA28JgvLZpd9YQSzt9T18B9WFGder4wQaZ7JNEl%2BTDA0S9tS4SRy%2BA3FuM%2FUHk9LzfipXQ6AOmTU2AVdCZ%2FQgpDoFtg7C8PiAUsURyRuFaNuhOxHJpBJtVxCF9cP%2FmrXmhXc48e%2B7BHE0lwDbaLBveWFJTtWbegJ818I%2FmOXXTPeRmX9w%3D%3D&response-content-disposition=attachment%3B+filename%3DtrainX.csv"
url_Y_train = "https://storage.googleapis.com/kagglesdsdata/competitions/20137/1112534/trainY.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1589791250&Signature=WkNQRZ%2BC9rK2HMhzAgncjSTdGuaSajz0z8JRztQ57GTupBulivzD0GKK5579rOqZaCQPoUpV2J0H%2F2WeQI4cfVI8lCjdd74yHMzetwweBQsDCoOc7iInbNlAdA3YpG%2Fr6oaz%2Bq9DsWxwkf%2FIL3LqYmaz4kx5vWnwrn5mfMto5GHrPy99AOMhLJw7ntq4ztD6wntW0XgjTQZ7qaH7KT9Mb9zNKyqsSWksvzznYzSwECPtFr5NIRRs3%2FjjbtEnSlNse%2F5Bp8nI%2B7QYft%2BED1b6c%2FyK%2FIEP%2BkOfUS1z8DqeiEzE2Vi%2Btxwlej%2BCVY7OYK9bXbC8nP5q8l%2BQWjQwIurB%2FQ%3D%3D&response-content-disposition=attachment%3B+filename%3DtrainY.csv"
url_X_test = "https://storage.googleapis.com/kagglesdsdata/competitions/20137/1112534/testX.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1589791195&Signature=cM1zqND7uTBwfMJ43jXiJADMAJmeXi%2B02xdmEuRa9C3p0yzLuPEGhFYTQ4XyTpvUkehQtpJUGq3h%2FwyNE%2F9B1mrM6QykOZHmFB1X2Iqe%2FS4cFoF%2BGQnsYmteaxwqNZsDjcq8FNcTwm9Fhc1qeN9XjFIIcr5nubWqnv%2FDk%2BTL4GwMH2dr8pk4JrW%2BAX3ebBcLycsEoifzwHWADmHURT%2FXrSHV0QwFftvurV9AH6mqNv7Y338ojcrZZgRx7juAPXHIefjwzkFJ0KcdYaE2WhOL8Tknh1LaGEXXzStZBXGg2pkFWUD8H49DiqXKUbhxdHl2YhFjZxpJvsjEocqfQS%2BZmQ%3D%3D&response-content-disposition=attachment%3B+filename%3DtestX.csv"

s_1 = requests.get(url_X_train).content
s_2 = requests.get(url_Y_train).content
s_3 = requests.get(url_X_test).content

X_train = pd.read_csv(io.StringIO(s_1.decode('utf-8')), index_col = 'Id').to_numpy()
Y_train = pd.read_csv(io.StringIO(s_2.decode('utf-8')), index_col = 'Id').Value.to_numpy()
X_test = pd.read_csv(io.StringIO(s_3.decode('utf-8')), index_col = 'Id').to_numpy()

Definition of PolynomialFeatures

In [0]:
poly = PolynomialFeatures(2)

# Hyperparameters fitting

---



In [0]:
X = poly.fit_transform(X_train)
elastic = ElasticNet(max_iter=10000)

distributions = {
    'alpha': uniform(),
    'l1_ratio': uniform(),
}

rand_search = RandomizedSearchCV(elastic, distributions, n_iter=10, cv=5)
search_result = rand_search.fit(X, Y_train)

print('Best parameters:', search_result.best_params_)
print('Best parameters:', search_result.best_score_)

elastic_rand_search_prm = search_result.best_params_

Best parameters: {'alpha': 0.10459015653167436, 'l1_ratio': 0.17594033536340237}
Best parameters: 0.24154627137117002


In [0]:
X = poly.fit_transform(X_train)
passive_aggressive = PassiveAggressiveRegressor(max_iter=5000, random_state=0)

distributions = {
    'C': uniform(loc=0, scale=0.1),
    'epsilon': uniform(loc=0, scale=0.1),
    'tol': uniform(1e-3, 1e-10),
}

rand_search = RandomizedSearchCV(passive_aggressive, distributions, n_iter=100, cv=5)
search_result = rand_search.fit(X, Y_train)

print('Best parameters:', search_result.best_params_)
print('Best parameters:', search_result.best_score_)

passive_aggressive_rand_search_prm = search_result.best_params_

Best parameters: {'C': 0.008766832563621152, 'epsilon': 0.03312366582829205, 'tol': 0.0010000000266718165}
Best parameters: 0.3151735931348718


In [0]:
X = poly.fit_transform(X_train)
ard = ARDRegression(n_iter=300)

distributions = {
    'tol': uniform(1e-2, 1e-5),
    'alpha_1': uniform(1e-1, 1e-11),
    'alpha_2': uniform(1e-1, 1e-11),
    'lambda_1': uniform(1e-1, 1e-11),
    'lambda_2': uniform(1e-1, 1e-11),
    'lambda_1': uniform(1e-1, 1e-11),
}

rand_search = RandomizedSearchCV(ard, distributions, n_iter=2, cv=5)
search_result = rand_search.fit(X, Y_train)

print('Best parameters:', search_result.best_params_)
print('Best parameters:', search_result.best_score_)

ard_rand_search_prm = search_result.best_params_

Best parameters: {'alpha_1': 0.10000000000097219, 'alpha_2': 0.1000000000021046, 'lambda_1': 0.1000000000053792, 'lambda_2': 0.1000000000082386, 'tol': 0.010004695408376181}
Best parameters: 0.3252779132921292


In [0]:
X = poly.fit_transform(X_train)
elastic_cv = ElasticNetCV(n_alphas=200, max_iter=5000)

distributions = {
    'l1_ratio': uniform(0, 1),
    'eps': uniform(1e-8, 1e-1),
    'tol': uniform(1e-8, 1e-3),
}

rand_search = RandomizedSearchCV(elastic_cv, distributions, n_iter=100, cv=2)
search_result = rand_search.fit(X, Y_train)

print('Best parameters:', search_result.best_params_)
print('Best parameters:', search_result.best_score_)

elastic_cv_rand_search_prm = search_result.best_params_

Best parameters: {'eps': 0.0042860872344182175, 'l1_ratio': 0.6355739860020118, 'tol': 0.0001601386836479555}
Best parameters: 0.3097474453445239


In [0]:
X = poly.fit_transform(X_train)
huber = HuberRegressor(max_iter=5000)

distributions = {
    'epsilon': uniform(1, 2),
    'alpha': uniform(1e-8, 1e-2),
    'tol': uniform(1e-8, 1e-3),
}

rand_search = RandomizedSearchCV(huber, distributions, n_iter=20, cv=5)
search_result = rand_search.fit(X, Y_train)

print('Best parameters:', search_result.best_params_)
print('Best parameters:', search_result.best_score_)

huber_rand_search_prm = search_result.best_params_

Best parameters: {'alpha': 0.009886786472622392, 'epsilon': 1.9115525243305342, 'tol': 0.0003906184668668611}
Best parameters: 0.3186498853235981


In [0]:
X = poly.fit_transform(X_train)


# prm = {
#     'C': 1.0,
#     'epsilon': 0.1,
#     'tol': 0.001,
# }

prm = np.array([1.0, 0.1, 0.001])
eps = 1e-2

def grad(prm, fixed):
    g = np.zeros(3)
    passive_aggressive = PassiveAggressiveRegressor(max_iter=5000, random_state=0, **{'C': prm[0] + 0.000001, 'epsilon': prm[1], 'tol': prm[2]})
    g[0] = (cross_val_score(passive_aggressive, X, Y_train, cv=5).mean() - fixed) / 0.000001
    
    passive_aggressive = PassiveAggressiveRegressor(max_iter=5000, random_state=0, **{'C': prm[0], 'epsilon': prm[1] + 0.000001, 'tol': prm[2]})
    g[1] = (cross_val_score(passive_aggressive, X, Y_train, cv=5).mean() - fixed) / 0.000001
    
    passive_aggressive = PassiveAggressiveRegressor(max_iter=5000, random_state=0, **{'C': prm[0], 'epsilon': prm[1], 'tol': prm[2] + 0.000001})
    g[2] = (cross_val_score(passive_aggressive, X, Y_train, cv=5).mean() - fixed) / 0.000001
    return g

for i in range(100):
    passive_aggressive = PassiveAggressiveRegressor(max_iter=5000, random_state=0, **{'C': prm[0], 'epsilon': prm[1], 'tol': prm[2]})
    score = cross_val_score(passive_aggressive, X, Y_train, cv=5).mean()
    print(score)
    prm = prm + grad(prm, score) * eps

passive_aggressive_rand_search_prm = {
    'C': prm[0],
    'epsilon': prm[1],
    'tol': prm[2],
}

# Final training and prediction

---



#### GradientBoostingRegressor
Kaggle public test score: 0.54379 👇

In [35]:
X = poly.fit_transform(X_train)
testX = poly.fit_transform(X_test)

params = {'n_estimators': 500,
          'max_depth': 4,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}

passive_aggressive = ensemble.GradientBoostingRegressor(**params)
passive_aggressive.fit(X, Y_train)
y_pred = passive_aggressive.predict(testX).reshape(2000,)

print(cross_val_score(passive_aggressive, X, Y_train, cv=5).mean())
y_pred

0.5061368591697395


array([26.91039649, 31.72797952, 28.74737119, ..., 26.97310358,
       30.96112832, 25.97880043])

#### PassiveAggressiveRegressor with custom gradient params fitting
Kaggle public test score: 0.65182 👇

In [32]:
X = poly.fit_transform(X_train)
testX = poly.fit_transform(X_test)

passive_aggressive = PassiveAggressiveRegressor(max_iter=7000, random_state=0, **passive_aggressive_rand_search_prm)
passive_aggressive.fit(X, Y_train)
y_pred = passive_aggressive.predict(testX).reshape(2000,)

print(cross_val_score(passive_aggressive, X, Y_train, cv=5).mean())
y_pred

-0.5918526845256971


array([22.14615099, 28.10751042, 24.76117155, ..., 26.23221528,
       31.29180287, 29.48728386])

#### PassiveAggressiveRegressor
Kaggle public test score: 0.64698 👇

In [0]:
X = poly.fit_transform(X_train)
testX = poly.fit_transform(X_test)

passive_aggressive = PassiveAggressiveRegressor(max_iter=100, random_state=0)
passive_aggressive.fit(X, Y_train)
y_pred = passive_aggressive.predict(testX).reshape(2000,)

print(cross_val_score(passive_aggressive, X, Y_train, cv=5).mean())
y_pred

-0.9277521496759714


array([19.72246023, 34.83825565, 29.16675255, ..., 26.77971688,
       32.18134777, 29.75120905])

#### LinearRegression
Kaggle public test score: 0.43746 👇

In [0]:
X = poly.fit_transform(X_train)
testX = poly.fit_transform(X_test)

elastic = ElasticNet(max_iter=5000, **elastic_rand_search_prm)
elastic.fit(X, Y_train)
y_pred = elastic.predict(testX).reshape(2000,)

print(cross_val_score(elastic, X, Y_train, cv=5).mean())
y_pred

0.32892066631831585


array([28.96368967, 32.62823264, 30.7020049 , ..., 28.18940336,
       29.11997998, 28.59157524])

#### Elastic Net,
Kaggle public test score: 0.46244 👇

In [0]:
X = poly.fit_transform(X_train)
testX = poly.fit_transform(X_test)

linear = LinearRegression()
linear.fit(X, Y_train)
y_pred = linear.predict(testX).reshape(2000,)

print(cross_val_score(linear, X, Y_train, cv=5).mean())
y_pred

0.30151623990826026


array([23.95507812, 32.609375  , 27.62890625, ..., 23.5703125 ,
       28.68359375, 27.41992188])

#### ARDRegression
Kaggle public test score: 0.45476 👇

In [0]:
X = poly.fit_transform(X_train)
testX = poly.fit_transform(X_test)

ard = ARDRegression(n_iter=50)
ard.fit(X, Y_train)
y_pred = ard.predict(testX).reshape(2000,)

print(cross_val_score(ard, X, Y_train, cv=5).mean())
y_pred

0.3067418448396757


array([25.66468194, 32.34430956, 28.30981349, ..., 25.59082425,
       28.62951537, 28.62479063])

#### ElasticNetCV
Kaggle public test score: 0.45422 👇

In [0]:
X = poly.fit_transform(X_train)
testX = poly.fit_transform(X_test)

elastic = ElasticNetCV(max_iter=5000, **elastic_cv_rand_search_prm)
elastic.fit(X, Y_train)
y_pred = elastic.predict(testX).reshape(2000,)

print(cross_val_score(elastic, X, Y_train, cv=5).mean())
y_pred

0.3820592494613194


array([27.22065663, 32.13629039, 29.60666946, ..., 27.1374192 ,
       28.68501572, 28.30467458])

#### HuberRegressor
Kaggle public test score: 0.45547 👇

In [0]:
X = poly.fit_transform(X_train)
testX = poly.fit_transform(X_test)

huber = HuberRegressor(max_iter=5000, **huber_rand_search_prm)
huber.fit(X, Y_train)
y_pred = huber.predict(testX).reshape(2000,)

print(cross_val_score(huber, X, Y_train, cv=5).mean())
y_pred

0.3186498853235981


array([23.70549709, 31.93082229, 28.21532003, ..., 23.08918567,
       28.59482197, 27.4107996 ])

#### Writing to file

In [0]:
Y_predicted = pd.DataFrame({'Value': y_pred, 'Id': range(len(y_pred))})
Y_predicted.to_csv('mullagaliev_amir.csv', index=False)